# Segementation and Clustering Neighborhoods in Toronto

## Week 3 for Applied Data Science Capstone Course

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

### Lets import the data from wikipedia (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### 1. Download Dataset https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [4]:
!wget -q -O 'toronto_data.json' 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
d=pd.read_html(url)

In [6]:
# find number of tables found on the webpage
len(d)

3

In [7]:
#store the 1st table on the webpage to the data frame
df_temp=d[0]
df_temp.shape

(180, 3)

In [8]:
df_temp.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Ignore cells with a borough that is Not assigned.

In [9]:
df = df_temp[df_temp['Borough'] != 'Not assigned'].reset_index().drop(columns='index')
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df.shape

(103, 3)

## Part 2. Get Latitude and longtiude of each nieghborhood

In [54]:
!pip install geocoder

In [57]:
import geocoder

def get_latlon(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

res = get_latlon('Hillcrest Village, North York, M2H Toronto, Ontario') # test function

In [59]:
lat=[]
longitude=[]
# for i in range(2):
for i in range(df.shape[0]):
    address=str(df['Neighborhood'][i] + ", "  + df['Borough'][i] + ", "+ df['Postal Code'][i] + ", "+ 'Canada')
    result = get_latlon(address)
#     print(address, " : " , ", ", result[0], ",", result[1])
    lat.append(result[0])
    longitude.append(result[1])

In [60]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [66]:
df['Latitude'] = lat

In [67]:
df['Longitude'] = longitude

In [68]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758872,-79.320292
1,M4A,North York,Victoria Village,43.731540,-79.314280
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.714880,-79.449933
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
